# Model Comparison Lab

In this lab we will compare the performance of all the models we have learned about so far, using the car evaluation dataset.

## 1. Prepare the data

The [car evaluation dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/car/) is in the assets/datasets folder. By now you should be very familiar with this dataset.

1. Load the data into a pandas dataframe
- Encode the categorical features properly: define a map that preserves the scale (assigning smaller numbers to words indicating smaller quantities)
- Separate features from target into X and y

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.grid_search import GridSearchCV

In [8]:
df = pd.read_csv("../../assets/datasets/car.csv")
df.head()

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [9]:
print(df.persons.unique())
print(df.buying.unique())
print(df.doors.unique())
print(df.safety.unique())
print(df.acceptability.unique())
print(df.maint.unique())
print(df.lug_boot.unique())

['2' '4' 'more']
['vhigh' 'high' 'med' 'low']
['2' '3' '4' '5more']
['low' 'med' 'high']
['unacc' 'acc' 'vgood' 'good']
['vhigh' 'high' 'med' 'low']
['small' 'med' 'big']


In [26]:
map1 = {"low":1, 
        "med":2, 
        "high":3, 
        "vhigh":4}
map2 = {"small":1, 
        "med":2,
        "big":3}
map3 = {"unacc":1, 
        "acc":2, 
        "good":3, 
        "vgood":4}
map4 = {"2":2,
        "4":4,
        "more":5}
map5 = {"2":2, 
        "3":3, 
        "4":4,
        "5more": 5}

In [28]:
features = [c for c in df.columns if c != "acceptability"]
dfn = df.copy()

dfn.buying = df.buying.map(map1)
dfn.maint = df.maint.map(map1)
dfn.lug_boot = df.lug_boot.map(map2)
dfn.persons = df.persons.map(map4)
dfn.doors = df.doors.map(map5)
dfn.safety = df.safety.map(map1)
dfn.acceptability = df.acceptability.map(map3)

X = dfn[features]
y = dfn['acceptability']
X.head()


,buying,maint,doors,persons,lug_boot,safety
0,4,4,2,2,1,1
1,4,4,2,2,1,2
2,4,4,2,2,1,3
3,4,4,2,2,2,1
4,4,4,2,2,2,2


In [29]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying           1728 non-null int64
maint            1728 non-null int64
doors            1728 non-null int64
persons          1728 non-null int64
lug_boot         1728 non-null int64
safety           1728 non-null int64
acceptability    1728 non-null int64
dtypes: int64(7)
memory usage: 94.6 KB


## 2. Useful preparation

Since we will compare several models, let's write a couple of helper functions.

1. Separate X and y between a train and test set, using 30% test set, random state = 42
    - make sure that the data is shuffled and stratified
2. Define a function called `evaluate_model`, that trains the model on the train set, tests it on the test, calculates:
    - accuracy score
    - confusion matrix
    - classification report
3. Initialize a global dictionary to store the various models for later retrieval


In [30]:
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,classification_report

In [60]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42, stratify =y)


def evaluate_model(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test,y_pred)
    
    cm= confusion_matrix(y_test,y_pred)
    cr = classification_report(y_test,y_pred)
    
    print cm
    print cr
    
    return a
all_models = {}

## 3.a KNN

Let's start with `KNeighborsClassifier`.

1. Initialize a KNN model
- Evaluate it's performance with the function you previously defined
- Find the optimal value of K using grid search
    - Be careful on how you perform the cross validation in the grid search

In [33]:
from sklearn.neighbors import KNeighborsClassifier
a = evaluate_model(KNeighborsClassifier())

[[354   9   0   0]
 [  8 107   0   0]
 [  0   9  11   1]
 [  0   2   0  18]]
             precision    recall  f1-score   support

          1       0.98      0.98      0.98       363
          2       0.84      0.93      0.88       115
          3       1.00      0.52      0.69        21
          4       0.95      0.90      0.92        20

avg / total       0.95      0.94      0.94       519



In [34]:
from sklearn.grid_search import GridSearchCV

params = {"n_neighbors" : range(2,60)}

gsknn = GridSearchCV(KNeighborsClassifier(),
                    params, n_jobs = -1,
                    cv = KFold(len(y), n_folds = 3, shuffle =True))

In [35]:
gsknn.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1728, n_folds=3, shuffle=True, random_state=None),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [23]:
gsknn.best_params_

{'n_neighbors': 5}

In [24]:
gsknn.best_score_

0.94618055555555558

In [63]:
gsknn.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [66]:
a= evaluate_model(gsknn.best_estimator_)

[[354   9   0   0]
 [  8 107   0   0]
 [  0   9  11   1]
 [  0   2   0  18]]
             precision    recall  f1-score   support

          1       0.98      0.98      0.98       363
          2       0.84      0.93      0.88       115
          3       1.00      0.52      0.69        21
          4       0.95      0.90      0.92        20

avg / total       0.95      0.94      0.94       519



In [62]:
all_models["knn"] = {"model": gsknn.best_estimator_,
                    "score" : a}

In [64]:
all_models

{'knn': {'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, n_jobs=1, n_neighbors=5, p=2,
             weights='uniform'), 'score': 0.77456647398843925}}

## 3.b Bagging + KNN

Now that we have found the optimal K, let's wrap `KNeighborsClassifier` in a BaggingClassifier and see if the score improves.

1. Wrap the KNN model in a Bagging Classifier
- Evaluate performance
- Do a grid search only on the bagging classifier params

In [68]:
from sklearn.ensemble import BaggingClassifier

In [69]:
import sklearn

In [70]:
sklearn.__version__

'0.17.1'

## Wrap the KNN model in a Bagging Classifier

In [110]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  BaggingClassifier
from sklearn.grid_search import GridSearchCV

cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)

knn = KNeighborsClassifier()
bgClass = BaggingClassifier(knn)
a = evaluate_model(bgClass)

[[353  10   0   0]
 [ 10 103   2   0]
 [  0   8  12   1]
 [  0   4   0  16]]
             precision    recall  f1-score   support

          1       0.97      0.97      0.97       363
          2       0.82      0.90      0.86       115
          3       0.86      0.57      0.69        21
          4       0.94      0.80      0.86        20

avg / total       0.93      0.93      0.93       519



In [111]:
bgClass.fit(X,y)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [112]:
bgClass.best_params_

AttributeError: 'BaggingClassifier' object has no attribute 'best_params_'

In [108]:
bgClass.best_score_

AttributeError: 'BaggingClassifier' object has no attribute 'best_score_'

In [ ]:
a= evaluate_model(bgClass.best_estimator_)

## Do a grid search only on the bagging classifier params

In [93]:
bagging_params ={"n_estimators": [10,20],
                "max_samples":[0.7,1.0],
                "max_features":[0.7,1.0],
                "bootstrap_features":[True,False]}
gsbaggingknn = GridSearchCV(bgClass,
                    bagging_params, n_jobs = -1,
                    cv = KFold(len(y), n_folds = 3, shuffle =True))

In [94]:
gsbaggingknn 

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1728, n_folds=3, shuffle=True, random_state=None),
       error_score='raise',
       estimator=BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20], 'max_samples': [0.7, 1.0], 'bootstrap_features': [True, False], 'max_features': [0.7, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [95]:
gsbaggingknn.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1728, n_folds=3, shuffle=True, random_state=None),
       error_score='raise',
       estimator=BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20], 'max_samples': [0.7, 1.0], 'bootstrap_features': [True, False], 'max_features': [0.7, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [87]:
gsbaggingknn.best_params_

{'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 0.7,
 'n_estimators': 20}

In [96]:
gsbaggingknn.best_score_

0.94965277777777779

In [97]:
gsbaggingknn.best_estimator_

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=20, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [98]:
a= evaluate_model(gsbaggingknn.best_estimator_)

[[356   7   0   0]
 [  4 110   1   0]
 [  0   7  14   0]
 [  0   3   1  16]]
             precision    recall  f1-score   support

          1       0.99      0.98      0.98       363
          2       0.87      0.96      0.91       115
          3       0.88      0.67      0.76        21
          4       1.00      0.80      0.89        20

avg / total       0.96      0.96      0.96       519



## 4. Logistic Regression

Let's see if logistic regression performs better

1. Initialize LR and test on Train/Test set
- Find optimal params with Grid Search
- See if Bagging improves the score

In [57]:
from sklearn.linear_model import LogisticRegression
a = evaluate_model(LogisticRegression())

[[347  11   4   1]
 [ 59  53   3   0]
 [  5  15   1   0]
 [  0  19   0   1]]
             precision    recall  f1-score   support

          1       0.84      0.96      0.90       363
          2       0.54      0.46      0.50       115
          3       0.12      0.05      0.07        21
          4       0.50      0.05      0.09        20

avg / total       0.73      0.77      0.74       519



In [ ]:
from sklearn.grid_search import GridSearchCV

params = {"n_neighbors" : range(2,60)}

gsknn = GridSearchCV(KNeighborsClassifier(),
                    params, n_jobs = -1,
                    cv = KFold(len(y), n_folds = 3, shuffle =True))

In [ ]:
gsknn.fit(X,y)

In [ ]:
bgknn.best_params_

In [ ]:
bgknn.best_score_

## 5. Decision Trees

Let's see if Decision Trees perform better

1. Initialize DT and test on Train/Test set
- Find optimal params with Grid Search
- See if Bagging improves the score

In [ ]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)

Now let's initialize a Decision Tree Classifier and evaluate its performance:

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')
bdt = BaggingClassifier(DecisionTreeClassifier())
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s.mean().round(3), s.std().round(3))
score(bdt, "Bagging DT")

## 6. Support Vector Machines

Let's see if SVM perform better

1. Initialize SVM and test on Train/Test set
- Find optimal params with Grid Search
- See if Bagging improves the score

## 7. Random Forest & Extra Trees

Let's see if Random Forest and Extra Trees perform better

1. Initialize RF and ET and test on Train/Test set
- Find optimal params with Grid Search

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
bdt = BaggingClassifier(DecisionTreeClassifier())
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

def score(model, name):
    s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

score(dt, "Decision Tree")
score(bdt, "Bagging DT")
score(rf, "Random Forest")
score(et, "Extra Trees")



## 8. Model comparison

Let's compare the scores of the various models.

1. Do a bar chart of the scores of the best models. Who's the winner on the train/test split?
- Re-test all the models using a 3 fold stratified shuffled cross validation
- Do a bar chart with errorbars of the cross validation average scores. is the winner the same?


## Bonus

We have encoded the data using a map that preserves the scale.
Would our results have changed if we had encoded the categorical data using `pd.get_dummies` or `OneHotEncoder`  to encode them as binary variables instead?

1. Repeat the analysis for this scenario. Is it better?
- Experiment with other models or other parameters, can you beat your classmates best score?